<a href="https://colab.research.google.com/github/koki0702/zerobook3/blob/master/notebook/ja/step07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 前ステップまでに実装したコード

In [20]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

***

# ステップ7 バックプロパゲーションの自動化

前ステップで、バックプロパゲーションを動かすことには成功しました。しかし、そこでは逆伝播の計算を手作業でコーディングする必要がありました。これが意味するのは、新しい計算を行うたびに、私たち自身の手で逆伝播用のコードを書く必要があるということです。たとえば@<img>{ch1/1-14}のようないくつかの計算グラフを考えた場合、それぞれの計算で逆伝播のためのコードを手作業で書かなければなりません。それはミスが起きる可能性があり、何よりも退屈な作業です。退屈なことはPythonにやらせましょう！

<br>![img](images/1-14.png)
<br>**図7-1** 様々な計算グラフの例（変数名は省略し、関数をクラス名で記す）<br><br>

次に進むべき道は、逆伝播の自動化です。より正確に言えば、通常の計算（順伝播）を行えば――たとえ、それがどのような計算であったとしても――その逆伝播が自動で行われる仕組みを作ることです。いよいよこれからが、Define-by-Runの核心に触れる場所になります！

<br><div class="alert alert-info">
<b>【ノート】</b> Define-by-Runとは、ディープラーニングで行う計算の「つながり」を、計算を行うタイミングで作る仕組みのことです。これは「動的計算グラフ」とも呼ばれます。Define-by-Runについて、またその利点については「コラム Define-by-Run」で詳しく説明します。
</div><br>

ところで、**図7-1**で示した計算グラフは、どれもが一直線に並んだ計算です。そのため、関数の順番をリストの形で保持すれば、後は、それを逆向きに辿ることで、逆伝播を自動で行えます。しかし、枝分かれした計算グラフや、変数が複数回利用されるような複雑な計算グラフに関しては、単純なリストによる手法は使えません。私たちのこれからの目標は、どれだけ複雑な計算グラフであったとしても、自動で逆伝播できる仕組みを構築することです。

<br><div class="alert alert-warning">
<b>【注意】</b> 実は、リストのデータ構造を工夫すれば、行った計算をリストに追加していくだけで、任意の計算グラフに対して逆伝播を正しく行うことも可能です。そのデータ構造は、Wengertリストと呼ばれます（または「テープ」とも呼ばれます）。本書では、Wengertリストについての説明は行いません。Wengertリストに興味のある方は文献<a href="https://dl.acm.org/doi/10.1145/355586.364791">[2]</a> <a href="http://www.cs.cmu.edu/~wcohen/10-605/notes/autodiff.pdf">[3]</a>などを参考にしてください。また、Wengertリストに対するDefine-by-Runの利点は、文献<a href="https://openreview.net/pdf?id=BJJsrmfCZ">[4]</a>などを参考にしてください。
</div>


## 7.2 逆伝播の自動化のために

逆伝播を自動化するにあたり、変数と関数の関係性について考えるところからスタートします。まずは、関数の視点から、つまりは「関数から変数はどのように見えるか」ということについて考えます。関数から見ると、変数は「入力」と「出力」として存在します。**図7-2**の左図に示すように、関数にとって変数は「入力された変数（`input`）」と「出力した変数（`output`）」として存在します（図の破線は参照を示します）。

<br>![img](images/1-15.png)
<br>**図7-2** 関数から見た変数との関係性（左図）と変数から見た関数との関係性（右図）<br><br>

続いて、変数の視点から関数はどう見えるでしょうか？ ここで注目したい点は、変数が関数によって「生み出される」ということです。言い換えると、変数にとって関数は「生みの親」です。つまりは、`creator`（生み出す者）となる存在です。もし生みの親である関数が存在しないのであれば、それは関数以外によって生み出された変数、たとえばユーザによって与えられた変数であると考えられます。

それでは、**図7-2**で表される関数と変数の「つながり」を私たちのコードに取り入れましょう。ここでは、その「つながり」を、通常の計算（順伝播）が行われるまさにそのタイミングで作ることにします。そのために、まずは`Variable`クラスに対して次のようにコードを追加します。

In [21]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None       # ★（追加したコード）

    def set_creator(self, func):  # ★
        self.creator = func       # ★

ここでは、`creator`というインスタンス変数を追加します。そして、`creator`を設定するためのメソッドを`set_creator`メソッドとして追加します。続いて、`Function`クラスに対して次のようにコードを追加します。

In [22]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)  # ★出力変数に生みの親を覚えさせる
        self.input = input
        self.output = output      # ★出力も覚える
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

順伝播の計算により`output`という`Variable`インスタンスが生成されます。このとき、その生成された`output`に対して、「私（関数である自分自身）が生みの親であること」を覚えさせます。これこそが、動的に「つながり」を作る仕掛けの核心部です。なお、ここでは次のステップを見据えて、インスタンス変数の`output`に出力を設定しています。

<br><div class="alert alert-info">
<b>【ノート】</b>DeZeroの動的計算グラフの仕組みは、実際の計算が行われるときに、変数という「箱」にその「つながり」を記録することによって行われます。同様のアプローチは、ChainerやPyTorchでも行われています。
</div><br>

このように、「つながり」を持った`Variable`と`Function`があれば、計算グラフを逆向きに辿ることができます。具体的なコードで示すと、次のようになります。

In [23]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆向きに計算グラフのノードを辿る
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x